In [3]:
import pandas as pd
import faiss
import numpy as np

In [4]:
videos_db = pd.read_json('data2.json').T
videos_db.head()

,categories,sub_categories,description,embeddings
video_0,"[Art & Creativity, Education & Knowledge]","[literature, writing, technology, history]","This video is about literature, writing, techn...","[0.016384885000000002, -0.0025806671, 0.028271..."
video_1,[Entertainment],"[music, tv shows]","This video is about music, and tv shows.","[-0.025938371, 0.0019344718000000002, 0.041684..."
video_2,[Art & Creativity],[theater],This video is about theater.,"[-0.025047548000000003, 0.017019922, 0.0198283..."
video_3,"[Lifestyle, Entertainment]","[home decor, memes]","This video is about home decor, and memes.","[-0.00896976, 0.014831336, 0.0099496255, -0.09..."
video_4,"[Misc & Social, Lifestyle]","[news, relationships, food]","This video is about news, relationships, and f...","[-0.020393768, -0.00030702800000000004, 0.0290..."


In [23]:
#step 1: to index data
# we need to find dimensions of the embedding, so lets pick some random embedding here
dimension = len(videos_db.iloc[2]['embeddings'])
#this comes out as 3072

#lets build index, we are gonna use simple l2 index which is basically  Euclidean distance. 
# It performs an exact, brute-force search by calculating the L2 distance between the query vector 
# and every vector in the database. 

index = faiss.IndexFlatL2(dimension)

for row in videos_db.itertuples():
    # our embedding are 1D, ie its a list of 3072 items. we need to convert it into a matrix/ 2d list of 1 x 3072
    np_arr = np.array(row.embeddings)
    np_arr_2d = np_arr.reshape(1,-1)
    index.add(np_arr_2d)

    
print(index.ntotal) #see how many are in our indexed db now ie index

500


In [24]:
x_emb = np.array( videos_db.iloc[2].embeddings)
x_emb_2d = x_emb.reshape(1,-1)
k = 5

 # sanity check, basically we are gonna take a vector which is already in the index, and see the similarity,
 # ideally if the vector is already present in the index, then we should get 0 as distance since its identical
dist, similar_item_index = index.search(x_emb_2d, 5)
print(f"Distances = {dist}\nIndices of Them in Index/DB = {similar_item_index}")

Distances = [[0. 0. 0. 0. 0.]]
Indices of Them in Index/DB = [[  2 237 281 389 414]]


In [25]:
# here as you can see we got 5 results and the top match is at 
# distance 0 => its identical and the index of that is 2, which is also the index at which it is in our videos_db

# lets see what the nearest items are about:
similar_item_index_list = similar_item_index.tolist()[0] # similar_item_index is a 2d nd array we need to get the 1d array

for i in similar_item_index_list:
    print( videos_db.iloc[i].to_dict())


{'categories': ['Art & Creativity'], 'sub_categories': ['theater'], 'description': 'This video is about theater.', 'embeddings': [-0.025047548000000003, 0.017019922, 0.019828342000000002, -0.10167717, -0.0054975073, 0.019529272, -0.0036484355, -0.0115283625, -0.008601786, 0.011778375, -0.0064519835, 0.013206802, -0.0014447954000000002, -0.0032978926, 0.12266639600000001, 0.0028253642, -0.014164923000000001, -0.0012116692000000001, 0.014877346000000001, 0.0011962891, -0.010045139, 0.0016805332, -0.013951055, -0.015273602, 0.017296834, -0.010536644000000001, 0.005885675, 0.01545767, 0.030996416000000002, 0.002171538, 0.00403181, 0.015955493, -0.0060734865, 0.018883936, 0.027705276, 0.016544715, -0.008666746000000001, -0.002174909, -0.01867773, 0.01422119, -0.0030822293000000002, -0.0074298736, -0.00018897020000000002, -0.008967251, -0.0068948558, -0.0016282102, 0.0061205877, -0.022796849, 0.013453303000000002, 0.014425746000000001, -0.01619819, 0.0020252643000000002, -0.020172117, -0.149

In [26]:
# as you can see the items seem pretty similar to each other.
# the item we did the search for, and the immediate similar item we got 
# have self care, productivity, philosophy in common. 
# now that index is ready, lets save it. 
# dist, similar_item_index = index.search(x_emb_2d, 5)
faiss.write_index(index, "index.faiss")

In [27]:
history_df = pd.read_json('/Users/ayushpatne/Downloads/does this make sense?/social media algo/social-media-algo-code/history.json').T
history_df.head()

,embeddings,interactions,score,view_duration
295,"[-0.015819123, -0.008584461, 0.009794935000000...","{'rewatch_count': 0, 'view_time': 0.01}",0.01,1.5337


In [ ]:
# IndexIVF
# this requires more parameters than the normal Flat Index. 
dimensions = 3072
num_clusters = 10 #some arbitary number, i chose 10 cos we have 500 videos, so on an avg 50 videos per centroid/per cluster

# we need a quantizer or basically an index to keep centroids and determine which partition a vector belongs to
quantizer = faiss.IndexFlatL2(dimensions)

index_v2 = faiss.IndexIVFFlat(quantizer, dimensions, num_clusters)

embeddings = []

for row in videos_db.itertuples():
    # our embedding are 1D, ie its a list of 3072 items. we need to convert it into a matrix/ 2d list of 500 x 3072
    embeddings.append(row.embeddings)

np_emb = np.array(embeddings).astype('float32')

# np_emb.shape

index_v2.train(np_emb) 
index_v2.add(np_emb)

In [12]:
videos_db.iloc[4]

categories                               [Misc & Social, Lifestyle]
sub_categories                          [news, relationships, food]
description       This video is about news, relationships, and f...
embeddings        [-0.020393768, -0.00030702800000000004, 0.0290...
Name: video_4, dtype: object

In [ ]:
x_emb = np.array( videos_db.iloc[4].embeddings)
x_emb_2d = x_emb.reshape(1,-1)
k = 5

dist, i = index_v2.search(x_emb_2d, k)
print(f"Distances = {dist}\nIndices of Them in Index/DB = {i}")

Distances = [[0.         0.12388569 0.12388569 0.12753543 0.15515578]]
Indices of Them in Index/DB = [[  4  22 369  19 289]]


In [13]:
for i in i[0]:
    print( videos_db.iloc[i].to_dict())

{'categories': ['Misc & Social', 'Lifestyle'], 'sub_categories': ['news', 'relationships', 'food'], 'description': 'This video is about news, relationships, and food.', 'embeddings': [-0.020393768, -0.00030702800000000004, 0.029071905000000002, -0.053360794, -0.0028470070000000004, 0.013436387000000001, -0.0043901154, 0.026524046000000003, 0.008750715000000001, 0.011293821, -0.0012990724, -0.019678969, 0.0035578967000000003, 0.011440921000000001, 0.07723108000000001, 0.0005948456, 0.014129793000000002, -0.03627031, 0.012963949, 0.028248794, -0.0027812830000000003, 0.0018342999, 0.01532083, -0.020143522, 0.0001858851, 0.0052394094000000006, 0.014627294, -0.0050268676, 0.020163216, 0.007355635, 0.034981064, 0.019445002000000003, 0.0008661199, 0.026569812, 0.014090508000000002, -0.0026368585, -0.009879481, 0.0035758845000000003, -0.026158769000000002, -0.005332154, -0.014515899, 0.032431222, 0.018540189000000002, -0.007613599800000001, 0.010964091, -0.014062108, -0.0049530067, -0.00111044

In [16]:
# trying higher number of k
x_emb = np.array( videos_db.iloc[4].embeddings)
x_emb_2d = x_emb.reshape(1,-1)
k = 10

dist, i = index_v2.search(x_emb_2d, k)
print(f"Video \n{videos_db.iloc[4]}\n")
print(f"Distances = {dist}\nIndices of Them in Index/DB = {i}")

for i in i[0]:
    print( videos_db.iloc[i].to_dict())


Video 
categories                               [Misc & Social, Lifestyle]
sub_categories                          [news, relationships, food]
description       This video is about news, relationships, and f...
embeddings        [-0.020393768, -0.00030702800000000004, 0.0290...
Name: video_4, dtype: object

Distances = [[0.         0.12388569 0.12388569 0.12753543 0.15515578 0.15524048
  0.15922126 0.22454357 0.2858926  0.31604165]]
Indices of Them in Index/DB = [[  4  22 369  19 289 353 241  37 232  58]]
{'categories': ['Misc & Social', 'Lifestyle'], 'sub_categories': ['news', 'relationships', 'food'], 'description': 'This video is about news, relationships, and food.', 'embeddings': [-0.020393768, -0.00030702800000000004, 0.029071905000000002, -0.053360794, -0.0028470070000000004, 0.013436387000000001, -0.0043901154, 0.026524046000000003, 0.008750715000000001, 0.011293821, -0.0012990724, -0.019678969, 0.0035578967000000003, 0.011440921000000001, 0.07723108000000001, 0.0005948456, 0.0

In [17]:
# trying even higher number of k
x_emb = np.array( videos_db.iloc[4].embeddings)
x_emb_2d = x_emb.reshape(1,-1)
k = 50

dist, i = index_v2.search(x_emb_2d, k)
print(f"Video \n{videos_db.iloc[4]}\n")
print(f"Distances = {dist}\nIndices of Them in Index/DB = {i}")

sim = []

for i in i[0]:
    sim.append( videos_db.iloc[i].to_dict())

pd.DataFrame(sim)

Video 
categories                               [Misc & Social, Lifestyle]
sub_categories                          [news, relationships, food]
description       This video is about news, relationships, and f...
embeddings        [-0.020393768, -0.00030702800000000004, 0.0290...
Name: video_4, dtype: object

Distances = [[0.         0.12388569 0.12388569 0.12753543 0.15515578 0.15524048
  0.15922126 0.22454357 0.2858926  0.31604165 0.32132584 0.34061176
  0.34088495 0.3424881  0.3424881  0.35504243 0.35780957 0.36912507
  0.40709692 0.41378132 0.44435322 0.4493343  0.45252556 0.45435375
  0.4547671  0.46225542 0.489593   0.4935762  0.49797103 0.49833748
  0.5027488  0.5057899  0.5119414  0.5150177  0.5224192  0.5224192
  0.533021   0.5352826  0.5374452  0.54179025 0.55104285 0.55354685
  0.5554635  0.56059086 0.5676311  0.569106   0.5760287  0.57684076
  0.58374786 0.58503044]]
Indices of Them in Index/DB = [[  4  22 369  19 289 353 241  37 232  58  87 381 473 340 377 119 304 322
  224 

,categories,sub_categories,description,embeddings
0,"[Misc & Social, Lifestyle]","[news, relationships, food]","This video is about news, relationships, and f...","[-0.020393768, -0.00030702800000000004, 0.0290..."
1,[Misc & Social],"[news, relationships]","This video is about news, and relationships.","[-0.03262524, 0.008164965, 0.017554559, -0.055..."
2,[Misc & Social],"[news, relationships]","This video is about news, and relationships.","[-0.03262524, 0.008164965, 0.017554559, -0.055..."
3,[Misc & Social],"[relationships, news]","This video is about relationships, and news.","[-0.032559574, 0.0032840592000000003, 0.016643..."
4,"[Misc & Social, Inspiration & Personal Growth]","[news, relationships, philosophy]","This video is about news, relationships, and p...","[-0.020836985000000002, 0.00040155400000000004..."
5,"[Entertainment, Misc & Social]","[music, relationships, news]","This video is about music, relationships, and ...","[-0.02646382, -0.0062212050000000005, 0.014567..."
6,"[Misc & Social, Education & Knowledge]","[news, relationships, finance]","This video is about news, relationships, and f...","[-0.011397271, -0.0055169896, 0.02384077200000..."
7,"[Trends & Modern Culture, Misc & Social]","[startups, relationships, news]","This video is about startups, relationships, a...","[-0.029367637000000002, 0.0007615588, 0.037632..."
8,"[Misc & Social, Entertainment]","[news, tv shows]","This video is about news, and tv shows.","[-0.028107857000000003, 0.0062972390000000005,..."
9,"[Misc & Social, Inspiration & Personal Growth]","[news, family, motivation]","This video is about news, family, and motivation.","[-0.02479461, 0.013481495000000001, 0.02872929..."


In [20]:
# trying even higher number of k, where k > estimated num of items per cluster
x_emb = np.array( videos_db.iloc[4].embeddings)
x_emb_2d = x_emb.reshape(1,-1)
k = 100

dist, i = index_v2.search(x_emb_2d, k)
print(f"Video \n{videos_db.iloc[4]}\n")
print(f"Indices of Them in Index/DB = {i}")

sim = []

for i in i[0]:
    sim.append( videos_db.iloc[i].to_dict())

pd.DataFrame(sim)

Video 
categories                               [Misc & Social, Lifestyle]
sub_categories                          [news, relationships, food]
description       This video is about news, relationships, and f...
embeddings        [-0.020393768, -0.00030702800000000004, 0.0290...
Name: video_4, dtype: object

Indices of Them in Index/DB = [[  4  22 369  19 289 353 241  37 232  58  87 381 473 340 377 119 304 322
  224 100 488  74 429  51 335 410 132 307 407  16  10 174 475 401 270 433
  273 378  66 463 469 114 141 342  38 447 436 424 115  97 344  68 101   9
  124  59 127 110 476  13  34 294 288  12 370 484 240  61  63 327 376  77
  430 103  88 159 209 395 262 419  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   -1  -1  -1  -1  -1  -1  -1  -1  -1  -1]]


,categories,sub_categories,description,embeddings
0,"[Misc & Social, Lifestyle]","[news, relationships, food]","This video is about news, relationships, and f...","[-0.020393768, -0.00030702800000000004, 0.0290..."
1,[Misc & Social],"[news, relationships]","This video is about news, and relationships.","[-0.03262524, 0.008164965, 0.017554559, -0.055..."
2,[Misc & Social],"[news, relationships]","This video is about news, and relationships.","[-0.03262524, 0.008164965, 0.017554559, -0.055..."
3,[Misc & Social],"[relationships, news]","This video is about relationships, and news.","[-0.032559574, 0.0032840592000000003, 0.016643..."
4,"[Misc & Social, Inspiration & Personal Growth]","[news, relationships, philosophy]","This video is about news, relationships, and p...","[-0.020836985000000002, 0.00040155400000000004..."
...,...,...,...,...
95,[Entertainment],[gaming],This video is about gaming.,"[-0.020674385, 0.0089926105, 0.02359821, -0.07..."
96,[Entertainment],[gaming],This video is about gaming.,"[-0.020674385, 0.0089926105, 0.02359821, -0.07..."
97,[Entertainment],[gaming],This video is about gaming.,"[-0.020674385, 0.0089926105, 0.02359821, -0.07..."
98,[Entertainment],[gaming],This video is about gaming.,"[-0.020674385, 0.0089926105, 0.02359821, -0.07..."


In [ ]:
#here the last few videos are just the same video -[1  -1  -1  -1  -1  -1  -1  -1  -1  -1
   # -1  -1  -1  -1  -1  -1  -1  -1  -1  -1] evident by the index
# by default default nprobe is 1, ie number of clusters to look in this index is 1
# lets try increasing it to 3
x_emb = np.array( videos_db.iloc[4].embeddings)
x_emb_2d = x_emb.reshape(1,-1)
k = 100

index_v2.nprobe = 3
dist, i = index_v2.search(x_emb_2d, k)
print(f"Video \n{videos_db.iloc[4]}\n")
print(f"Indices of Them in Index/DB = {i}")

sim = []

for i in i[0]:
    sim.append( videos_db.iloc[i].to_dict())

pd.DataFrame(sim)

Video 
categories                               [Misc & Social, Lifestyle]
sub_categories                          [news, relationships, food]
description       This video is about news, relationships, and f...
embeddings        [-0.020393768, -0.00030702800000000004, 0.0290...
Name: video_4, dtype: object

Indices of Them in Index/DB = [[  4  22 369  19 289 353 241  37 232  58  87 381 473 340 377 119 248 304
  322 255 286 338 399 224 100 260 236 104 207 446 206 488 264  74 429  51
  335  99 410 362  28  30 391 190 229 132 307 407  16  10 174 475 401 155
  270 433  80  49 273 378  73  96  66   1 463 357 297 406 329 252 142 469
  412 114 141 342 478 140 107 315 200 150 411 205 380  38 456 447  92  43
   44 212 247 277 420 436 424 225 115  97]]


,categories,sub_categories,description,embeddings
0,"[Misc & Social, Lifestyle]","[news, relationships, food]","This video is about news, relationships, and f...","[-0.020393768, -0.00030702800000000004, 0.0290..."
1,[Misc & Social],"[news, relationships]","This video is about news, and relationships.","[-0.03262524, 0.008164965, 0.017554559, -0.055..."
2,[Misc & Social],"[news, relationships]","This video is about news, and relationships.","[-0.03262524, 0.008164965, 0.017554559, -0.055..."
3,[Misc & Social],"[relationships, news]","This video is about relationships, and news.","[-0.032559574, 0.0032840592000000003, 0.016643..."
4,"[Misc & Social, Inspiration & Personal Growth]","[news, relationships, philosophy]","This video is about news, relationships, and p...","[-0.020836985000000002, 0.00040155400000000004..."
...,...,...,...,...
95,"[Entertainment, Trends & Modern Culture]","[tv shows, anime, startups]","This video is about tv shows, anime, and start...","[-0.027558077, -0.0053898883, 0.05574140700000..."
96,"[Education & Knowledge, Trends & Modern Culture]","[psychology, cars, gadgets]","This video is about psychology, cars, and gadg...","[-0.0009405938, -0.0028734480000000002, 0.0405..."
97,[Entertainment],"[tv shows, dance]","This video is about tv shows, and dance.","[-0.020636525000000003, 0.008492527, 0.0460269..."
98,"[Trends & Modern Culture, Lifestyle]","[startups, AI, photography, fitness]","This video is about startups, AI, photography,...","[-0.016899247000000003, 0.020242691, 0.0411238..."


In [ ]:
# here we go, now we can have a more varied result

In [23]:
# saving the index
faiss.write_index(index_v2, "index_v2.faiss")